In [1]:
import pytorch_lightning as pl
from  data.data_loader import MyDataModule 
from training.PL_train import Main_Loop
import torch.optim as optim
import torchio as tio
import torch

In [2]:
size = (48, 60, 48)
model = "OneLayer"
criterion = "Jacard"
type_list = ["t1"]
prepare = False
epochs = 100

train_transformer = tio.Compose([
    tio.RandomMotion(p=0.2),
    tio.RandomBiasField(p=0.3),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.RandomNoise(p=0.5),
    tio.RandomFlip(),
    tio.OneOf({
        tio.RandomAffine(): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
])

val_transformer = tio.Compose([
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
])

In [3]:

data_module = MyDataModule(
    train_transformer=train_transformer,
    val_transformer = val_transformer,
    prepare=prepare,
    size = size,
    type_list = type_list,
    sample_list= type_list
)

trainer = pl.Trainer(gpus=1,max_epochs = epochs)
main = Main_Loop(model = model,loss= criterion,type_list = type_list,scheduler = optim.lr_scheduler.CosineAnnealingLR,scheduler_args = {"T_max":100})
trainer.fit(main, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | OneLayer   | 10    
1 | loss  | JacardLoss | 0     
-------------------------------------
10        Trainable params
0         Non-trainable params
10        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/ludauter/anaconda3/envs/project/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [7]:
preprocessing_transform = tio.Compose([
    tio.ToCanonical(),
    tio.Resample(4),
    tio.CropOrPad(size),
])

In [ ]:
dataset = tio.datasets.ixi.IXI(root = "/home/ludauter/Documents/IXI",download=True,modalities  = ["T1","T2"],transform=preprocessing_transform)

dataset = tio.datasets.episurg.EPISURG(root = "/home/ludauter/Documents/episurg", transform=preprocessing_transform).get_paired()

In [ ]:
train_loader = torch.utils.data.DataLoader(
                dataset,
                batch_size=4,
                shuffle=False,
                num_workers=2,
            )

In [ ]:
for batch in dataset:
    pass

RuntimeError: As described above, some images in the subject are not in the same space. You probably can use the transforms ToCanonical and Resample to fix this, as explained at https://github.com/fepegar/torchio/issues/647#issuecomment-913025695